## Executa os notebooks com as funções à serem utilizadas do 'utils'

In [59]:
%run ../utils/_vocabulary.ipynb

In [88]:
%run ../utils/_gold_indexing.ipynb

In [86]:
%run ../utils/_rename.ipynb

In [78]:
%run ../utils/_extract_text.ipynb

In [95]:
%run ../utils/_execute_annif.ipynb

## Carrega o Vocabulário do arquivo XML

In [61]:
nal_vocabulary = Vocabulary()
nal_vocabulary.loadFromXmlFile('./raw-files/vocabulary/nalt-full_dwn_20240716.rdf', 'en', VocabularyXMLFileType.SKOS)
#nal_vocabulary.generateTSVFile('./prepared-files/vocabulary/nalt-full_dwn_20240716.tsv')

''

## Carrega o gold indexing de treinamento e gera os arquivos '.key'

In [89]:
training_gold_indexing = GoldIndexing('./raw-files/gold-indexing/Gold_indexing_NAL_Agriculture_corpus_entrenaimento_8000.txt')
training_gold_indexing.usePreferentialLabelsFrom(nal_vocabulary)
#training_gold_indexing.generateTextFile('./prepared-files/gold-indexing/nal-training-gi.txt')
training_gold_indexing.generateKeyFiles('./prepared-files/corpus-training/')

## Extrai o texto dos arquivos para o treinamento

In [76]:
generateOutputFilesFromDirectory('./raw-files/corpus-training/Corpus_Agricultura_para_Entrenaimento_8000_documentos_TI-RE-PC-RF/', './prepared-files/corpus-training/')

## Extrai o texto dos arquivos para Teste

In [81]:
generateOutputFilesFromDirectory('./raw-files/corpus-test/Corpus_Agricultura_para_Test_1_500_documentos/', './prepared-files/corpus-test/Corpus_Agricultura_para_Test_1_500_documentos/')

In [ ]:
generateOutputFilesFromDirectory('./raw-files/corpus-test/Corpus_Agricultura_para_Test_2_500_documentos/', './prepared-files/corpus-test/Corpus_Agricultura_para_Test_2_500_documentos/')

In [ ]:
generateOutputFilesFromDirectory('./raw-files/corpus-test/Corpus_Agricultura_para_Test_3_500_documentos/', './prepared-files/corpus-test/Corpus_Agricultura_para_Test_3_500_documentos/')

## Renomeia os arquivos de teste de 1 à 500

In [82]:
valor_controle = {
    "Corpus_Agricultura_para_Test_1_500_documentos/" : 8000,
    "Corpus_Agricultura_para_Test_2_500_documentos/": 8500,
    "Corpus_Agricultura_para_Test_3_500_documentos/": 9000,
}

base_path = './prepared-files/corpus-test/'

for k, v in valor_controle.items():
    renameFilesFromDirectory(base_path + k, v)

## Renomeia os rótulos de gold indexing de 1 à 500

In [87]:
arquivos = {
    'Gold_indexing_NAL_Agricultura_corpus_para_Test_1' : 8000,
    'Gold_indexing_NAL_Agricultura_corpus_para_Test_2' : 8500,
    'Gold_indexing_NAL_Agricultura_corpus_para_Test_3' : 9000
}

base_input_path = './raw-files/gold-indexing/'
base_output_path = './prepared-files/gold-indexing/'

for k, v in arquivos.items():
    renameKeys(base_input_path + k + '.txt', base_output_path + k + '.txt', v)

## Carrega o volabulário nal no Annif

In [92]:
import subprocess

annif_vocab = 'nal'
out = subprocess.run('annif load-vocab ' + annif_vocab + ' ./raw-files/vocabulary/nalt-full_dwn_20240716.rdf', stdout=subprocess.PIPE, shell=True)
print(out.stdout)

creating subject index
saving vocabulary into SKOS file data/vocabs/nal/subjects.ttl


b'Loading vocabulary from SKOS file ./raw-files/vocabulary/nalt-full_dwn_20240716.rdf...\n'


## Realiza os treinamentos definidos no arquivo de projetos

In [98]:
annif_projects = [
    'tfidf-nal-en',
    'fasttext-nal-en',
    'omikuji-parabel-nal-en',
    'mllm-nal-en',
    'ensemble-nal-en'
]

for p in annif_projects:
    out = subprocess.run('annif train ' + p + ' ./prepared-files/corpus-training/', stdout=subprocess.PIPE, shell=True)
    print(out.stdout)

Backend tfidf: transforming subject corpus
Backend tfidf: creating vectorizer
Backend tfidf: creating similarity index


b''


Backend fasttext: creating fastText training file
Backend fasttext: creating fastText model
Read 1M words
Number of words:  45066
Number of labels: 9999
Progress: 100.0% words/sec/thread: 1987902 lr:  0.000000 avg.loss:  7.259676 ETA:   0h 0m 0s


b''


Backend omikuji: creating vectorizer
Backend omikuji: creating train file
30759 / 30759 [============================================] 100.00 % 1697.67/s -] 33.85 % 32115.91/s 0s 

b'2025-01-30T19:49:36.994Z \x1bINFO \x1b [omikuji::data] Loading data from data/projects/omikuji-parabel-nal-en/omikuji-train.txt\n2025-01-30T19:49:37.009Z \x1bINFO \x1b [omikuji::data] Parsing data\n2025-01-30T19:49:37.045Z \x1bINFO \x1b [omikuji::data] Loaded 7971 examples; it took 0.05s\n2025-01-30T19:49:37.051Z \x1bINFO \x1b [omikuji::model::train] Training model with hyper-parameters HyperParam { n_trees: 3, min_branch_size: 100, max_depth: 20, centroid_threshold: 0.0, collapse_every_n_layers: 0, linear: HyperParam { loss_type: Hinge, eps: 0.1, c: 1.0, weight_threshold: 0.1, max_iter: 20 }, cluster: HyperParam { k: 2, balanced: true, eps: 0.0001, min_size: 2 }, tree_structure_only: false, train_trees_1_by_1: false }\n2025-01-30T19:49:37.051Z \x1bINFO \x1b [omikuji::model::train] Initializing tree trainer\n2025-01-30T19:49:37.052Z \x1bINFO \x1b [omikuji::model::train] Computing label centroids\n2025-01-30T19:49:37.917Z \x1bINFO \x1b [omikuji::model::train] Start training forest\n20

Backend mllm: starting train
Backend mllm: preparing training data
Backend mllm: training model
Backend mllm: saving model


b''


Backend nn_ensemble: creating NN ensemble model
Backend nn_ensemble: Initializing source projects: tfidf-nal-en, fasttext-nal-en, mllm-nal-en
Backend nn_ensemble: Processing training documents...
Backend nn_ensemble: Training neural network model...
/home/marco/Projects/annif-experiments/.annif-venv/lib64/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


b'Epoch 1/10\n\r\x1b  1/250\x1b \x1b\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\x1b \x1b6:15\x1b 2s/step - loss: 5.9284e-04 - top_k_categorical_accuracy: 0.2188\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r\x1b  2/250\x1b \x1b\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\x94\x81\xe2\